In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import pickle
import napari
import numpy as np
import pandas as pd

from skimage.io import imread

import btrack
from btrack.constants import BayesianUpdates

sys.path.append('../libraries')
import input_functions as inp_f

In [79]:
info_file_path = r'Z:\Garrett\Livecell\022725_Paracrine_Senescence_48hr\slices\B03_single_channel_movie\downscaled\DIC_movie\data\info_B03.txt'

In [80]:
# read the file
info_file = open(info_file_path, 'r')
info_lines = info_file.readlines()
info_file.close()

# read info about the data frame
exp_dir,df_name = inp_f.read_df_info(info_lines)

df_dir = os.path.join(exp_dir,'df')
save_dir = df_dir

frames_to_exclude = inp_f.read_frames_2_exclude(info_lines)
#frames_to_exclude = eval(frames_to_exclude)

modelPath = os.path.join(exp_dir,'code','libraries','cell_config.json')

## Read in the data frame objects data

In [ ]:
data_df = pd.read_pickle(os.path.join(df_dir,df_name))

In [ ]:
len(data_df)

In [ ]:
data_df.columns

In [ ]:
# create a structure suitable for tracking

# choose objects 
sel_vector = [not(x in frames_to_exclude) for x in data_df.t]

objects_gen = data_df.loc[sel_vector,['label','area','centroid-1','centroid-0','major_axis_length','minor_axis_length','t']]

objects_gen.columns=['ID', 'area', 'x', 'y', 'major_axis_length','minor_axis_length','t']
objects_gen['z']=0
objects_gen['label']=5
objects_gen['prob']=0
objects_gen['dummy']=False
objects_gen['states']=0

#objects_gen.head()

In [ ]:
len(objects_gen)

In [ ]:
objects_gen.columns

## Tracking proper

In [ ]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file(modelPath)
    
    # approximate
    tracker.update_method = BayesianUpdates.EXACT
    tracker.max_search_radius = 100

    # append the objects to be tracked
    tracker.append(objects_gen)

    # set the volume (Z axis volume is set very large for 2D data)
    #tracker.volume=((0, data_df.size_x[0]), (0, data_df.size_y[0]), (-1e5, 1e5))
    tracker.volume = (
        (0, data_df["size_x"].iloc[0]),
        (0, data_df["size_y"].iloc[0]),
        (-1e5, 1e5)
)



    # track them (in interactive mode)
    #tracker.track_interactive(step_size=100)
    tracker.track()

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    # optional: get the data in a format for napari
    data, properties, graph = tracker.to_napari(ndim=2)
    # pickle Napari data
    with open(os.path.join(df_dir,'track.pkl'),'wb') as f:
        pickle.dump([data,properties,graph],f)

## Merging objects and tracking information

In [ ]:
trackDataAll = pd.DataFrame(data,columns=['track_id','t','x','y'])
trackDataAll['parent'] = properties['parent']
trackDataAll['generation'] = properties['generation']
trackDataAll['root'] = properties['root']

In [ ]:
len(trackDataAll)

In [ ]:
trackDataAll.columns

In [ ]:
allData = pd.merge(left=data_df,right=trackDataAll,left_on=['centroid-0','centroid-1','t'],right_on=['x','y','t'],how='left')

In [ ]:
allData.columns

In [ ]:
print(f'Number of all objects: {len(allData)}')

In [ ]:
# check how many objects doesn't have a track_id
test = np.sum(allData.track_id!=allData.track_id)
print(f'Number of objects without track_id: {test}')

## Be careful!!!

In [ ]:
# consider removing
#allData = allData.loc[allData.track_id==allData.track_id,:]
#print(f'Number of all objects: {len(allData)}')

## Define promising tracks

This part is manual at the moment.

In [ ]:
my_tracks = set(allData.track_id)
print(len(my_tracks))

In [ ]:
allData['accepted'] = False
allData['rejected'] = False
allData['promise'] = False

# Assuming track_lengths_to_test is a list of track lengths you want to test
track_lengths_to_test = [0, 1, 5, 10, 15, 20, 30, 50, 60]  # Example thresholds

# Initialize a dictionary to hold the count of tracks exceeding each length
tracks_exceeding_length = {length: 0 for length in track_lengths_to_test}

for track in set(allData.track_id):
    # Prepare signals for this track
    sel_signal = allData.loc[allData.track_id == track, ['t', 'mean_intensity-0_nuc', 'mean_intensity-0_ring']]
    sel_signal.sort_values(by='t', inplace=True)
    
    track_len = len(sel_signal)
    
    # Update the count for each threshold exceeded
    for length in track_lengths_to_test:
        if track_len > length:
            tracks_exceeding_length[length] += 1

# Report on the number of tracks exceeding each length threshold
for length, count in tracks_exceeding_length.items():
    print(f"Number of tracks longer than {length}: {count}")


In [ ]:
specific_length = 10  # Set based on results from previous block
promise_list = []

for track in set(allData.track_id):
    # Prepare signals for this track
    sel_signal = allData[allData.track_id == track]
    sel_signal.sort_values(by='t', inplace=True)
    
    if len(sel_signal) > specific_length:
        allData.loc[allData.track_id == track, 'promise'] = True
        promise_list.append(track)

print(f"Number of promising tracks longer than {specific_length}: {len(promise_list)}")


In [ ]:
len(promise_list)

## Create columns for requested annotations

In [ ]:
# get info about the tags (for annotating points on the tracks)
flag_list = inp_f.read_flags(info_lines,df=allData)

for flag in flag_list:
    
    allData[flag['flag_column']]=False

In [ ]:
# save df
allData.to_pickle(os.path.join(df_dir,df_name))
allData.to_csv(os.path.join(df_dir,df_name.replace('pkl','csv')),index=False)